In [ ]:
!pip install python-igraph
!pip install python-igraph leidenalg cairocffi
!pip install geopandas
!pip install folium matplotlib mapclassify

In [2]:
import pandas as pd
import numpy as np
import sys
from google.colab import files
import io
import matplotlib.pyplot as plt
import igraph as ig

import geopandas
import shapely
import folium
import json

In [3]:
# Read the file
uploaded = files.upload()

Saving traffic-signal-vehicle-and-pedestrian-volumes-data.xlsx to traffic-signal-vehicle-and-pedestrian-volumes-data.xlsx


In [4]:

my_df = pd.read_excel(io.BytesIO(uploaded['traffic-signal-vehicle-and-pedestrian-volumes-data.xlsx']))
my_df.head(5)

,TCS #,Main,Midblock Route,Side 1 Route,Side 2 Route,Activation Date,Latitude,Longitude,Count Date,8 Peak Hr Vehicle Volume,8 Peak Hr Pedestrian Volume
0,2,JARVIS ST,NaN,FRONT ST E,NaN,11/15/1948,43.649418,-79.371446,2017-06-21,15662,13535
1,3,KING ST E,NaN,JARVIS ST,NaN,08/23/1950,43.650461,-79.371924,2016-09-17,12960,7333
2,4,JARVIS ST,NaN,ADELAIDE ST E,NaN,09/12/1958,43.651534,-79.372360,2016-11-08,17770,7083
3,5,JARVIS ST,NaN,RICHMOND ST E,NaN,04/21/1962,43.652718,-79.372824,2015-12-08,19678,4369
4,6,JARVIS ST,NaN,QUEEN ST E,NaN,08/24/1928,43.653704,-79.373238,2016-09-17,14487,3368


In [5]:
uploaded = files.upload()
collision_df = pd.read_csv(io.BytesIO(uploaded['Motor Vehicle Collisions with KSI Data - 4326.csv']))

Saving Motor Vehicle Collisions with KSI Data - 4326.csv to Motor Vehicle Collisions with KSI Data - 4326.csv


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (42) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [6]:
my_df.sort_values(['8 Peak Hr Vehicle Volume','8 Peak Hr Pedestrian Volume'], ascending=[False,False])
DF= my_df.loc[:,["TCS #","Main","Side 1 Route","Side 2 Route","Latitude","Longitude","8 Peak Hr Vehicle Volume", "8 Peak Hr Pedestrian Volume" ]]
DF.head(5)


,TCS #,Main,Side 1 Route,Side 2 Route,Latitude,Longitude,8 Peak Hr Vehicle Volume,8 Peak Hr Pedestrian Volume
0,2,JARVIS ST,FRONT ST E,NaN,43.649418,-79.371446,15662,13535
1,3,KING ST E,JARVIS ST,NaN,43.650461,-79.371924,12960,7333
2,4,JARVIS ST,ADELAIDE ST E,NaN,43.651534,-79.372360,17770,7083
3,5,JARVIS ST,RICHMOND ST E,NaN,43.652718,-79.372824,19678,4369
4,6,JARVIS ST,QUEEN ST E,NaN,43.653704,-79.373238,14487,3368


In [7]:
DF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2280 entries, 0 to 2279
Data columns (total 8 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   TCS #                        2280 non-null   int64  
 1   Main                         2280 non-null   object 
 2   Side 1 Route                 2280 non-null   object 
 3   Side 2 Route                 758 non-null    object 
 4   Latitude                     2280 non-null   float64
 5   Longitude                    2280 non-null   float64
 6   8 Peak Hr Vehicle Volume     2280 non-null   int64  
 7   8 Peak Hr Pedestrian Volume  2280 non-null   int64  
dtypes: float64(2), int64(3), object(3)
memory usage: 142.6+ KB


In [8]:
gdf = geopandas.GeoDataFrame(
    DF, geometry=geopandas.points_from_xy(DF.Longitude, DF.Latitude, crs="epsg:4326"))

In [9]:
gdf.head()

,TCS #,Main,Side 1 Route,Side 2 Route,Latitude,Longitude,8 Peak Hr Vehicle Volume,8 Peak Hr Pedestrian Volume,geometry
0,2,JARVIS ST,FRONT ST E,NaN,43.649418,-79.371446,15662,13535,POINT (-79.37145 43.64942)
1,3,KING ST E,JARVIS ST,NaN,43.650461,-79.371924,12960,7333,POINT (-79.37192 43.65046)
2,4,JARVIS ST,ADELAIDE ST E,NaN,43.651534,-79.372360,17770,7083,POINT (-79.37236 43.65153)
3,5,JARVIS ST,RICHMOND ST E,NaN,43.652718,-79.372824,19678,4369,POINT (-79.37282 43.65272)
4,6,JARVIS ST,QUEEN ST E,NaN,43.653704,-79.373238,14487,3368,POINT (-79.37324 43.65370)


In [10]:
totalVolume = gdf['8 Peak Hr Vehicle Volume']+ gdf['8 Peak Hr Pedestrian Volume']
totalVolume
median= np.median(totalVolume)
min= np.amin(totalVolume)
min_mean = np.mean([min,median])


collision_df= collision_df.loc[:,["ACCLASS","IMPACTYPE","INVTYPE","INJURY","PEDESTRIAN","CYCLIST","AUTOMOBILE","geometry" ]]
collision_df

,ACCLASS,IMPACTYPE,INVTYPE,INJURY,PEDESTRIAN,CYCLIST,AUTOMOBILE,geometry
0,Fatal,Pedestrian Collisions,Driver,None,Yes,NaN,Yes,"{'type': 'Point', 'coordinates': (-79.45249, 4..."
1,Fatal,Pedestrian Collisions,Pedestrian,Fatal,Yes,NaN,Yes,"{'type': 'Point', 'coordinates': (-79.45249, 4..."
2,Fatal,Turning Movement,Motorcycle Driver,Fatal,NaN,NaN,Yes,"{'type': 'Point', 'coordinates': (-79.199786, ..."
3,Fatal,Turning Movement,Driver,None,NaN,NaN,Yes,"{'type': 'Point', 'coordinates': (-79.199786, ..."
4,Fatal,Pedestrian Collisions,Driver,None,Yes,NaN,Yes,"{'type': 'Point', 'coordinates': (-79.25619, 4..."
...,...,...,...,...,...,...,...,...
16855,Non-Fatal Injury,Cyclist Collisions,Cyclist,Major,NaN,Yes,Yes,"{'type': 'Point', 'coordinates': (-79.238926, ..."
16856,Non-Fatal Injury,Turning Movement,Driver,None,NaN,NaN,Yes,"{'type': 'Point', 'coordinates': (-79.232021, ..."
16857,Non-Fatal Injury,Turning Movement,Driver,Minor,NaN,NaN,Yes,"{'type': 'Point', 'coordinates': (-79.232021, ..."
16858,Non-Fatal Injury,Turning Movement,Passenger,Major,NaN,NaN,Yes,"{'type': 'Point', 'coordinates': (-79.232021, ..."


In [11]:
collision_df['geometry'] = collision_df['geometry'].map(lambda x: dict(eval(x)))

In [12]:
transformDF = collision_df['geometry'].apply(pd.Series)
coordinates = [x for x in transformDF.coordinates]


In [13]:
f = lambda x,index:tuple( i[index] for i in x)
gdf_collision = geopandas.GeoDataFrame(
    collision_df, geometry=geopandas.points_from_xy(f(coordinates,0),f(coordinates,1), crs="epsg:4326"))

In [40]:
gdf_collision.head(2)

,ACCLASS,IMPACTYPE,INVTYPE,INJURY,PEDESTRIAN,CYCLIST,AUTOMOBILE,geometry
0,Fatal,Pedestrian Collisions,Driver,None,Yes,NaN,Yes,POINT (-79.45249 43.65635)
1,Fatal,Pedestrian Collisions,Pedestrian,Fatal,Yes,NaN,Yes,POINT (-79.45249 43.65635)


In [41]:
gdf.head(2)

,TCS #,Main,Side 1 Route,Side 2 Route,Latitude,Longitude,8 Peak Hr Vehicle Volume,8 Peak Hr Pedestrian Volume,geometry
0,2,JARVIS ST,FRONT ST E,NaN,43.649418,-79.371446,15662,13535,POINT (-79.37145 43.64942)
1,3,KING ST E,JARVIS ST,NaN,43.650461,-79.371924,12960,7333,POINT (-79.37192 43.65046)


In [43]:
# Rename 'index' columns to avoid FutureWarning
dfp = gdf.rename(columns={'index': 'point'})
dfa = gdf_collision.rename(columns={'index': 'points'})

# Find points within polygons
out = geopandas.sjoin(dfp, dfa)

# Reduce rows
out = out.groupby('point') \
         .agg({'area': lambda x: x.tolist() if x.any() else [],
               'index_right': lambda x: dfa.loc[x, 'geometry'].tolist()
                                            if ~x.all() else []}) \
         .reset_index()

# Append columns
dfp = dfp.merge(out, on='point')

ImportError: ignored

In [44]:
gdf['riskLevel'] = np.where(totalVolume >= median, 2, np.where(totalVolume >= min_mean,1,0))
data=gdf.query("riskLevel == 2")
len(data)

1140

In [ ]:
result = data.to_json()
parsed = json.loads(result)
json_string= json.dumps(parsed, indent=0)
with open('json_data.json', 'w') as outfile:
    outfile.write(json_string)

In [ ]:
toronto_poly = shapely.wkt.loads(
    "POLYGON ((20.2213764 44.9154621, 20.2252450 44.9070062, 20.2399466 44.9067193, 20.2525385 44.8939145, 20.2419942 44.8842235, 20.2610016 44.8826597, 20.2794675 44.8754192, 20.2858284 44.8447802, 20.2856918 44.8332410, 20.3257447 44.8342507, 20.3328068 44.8098272, 20.3367239 44.8080890, 20.3339619 44.8058144, 20.3353253 44.8011005, 20.3336310 44.8003791, 20.3360230 44.7898245, 20.3384687 44.7907875, 20.3405086 44.7859144, 20.3417344 44.7872272, 20.3474466 44.7713203, 20.3509860 44.7687822, 20.3398029 44.7558716, 20.3220093 44.7448572, 20.3160895 44.7387338, 20.3235092 44.7345531, 20.3359605 44.7308053, 20.3437350 44.7301552, 20.3450306 44.7243651, 20.3497410 44.7209764, 20.3521450 44.7143627, 20.3633795 44.7046060, 20.3830709 44.7030441, 20.3845248 44.7011631, 20.3847991 44.7032182, 20.3924066 44.7036702, 20.4038881 44.6984458, 20.4097684 44.6992834, 20.4129839 44.7024603, 20.4192098 44.7021308, 20.4217436 44.7034920, 20.4251744 44.6976337, 20.4279418 44.6980838, 20.4313251 44.6940680, 20.4358368 44.6933579, 20.4402665 44.6905161, 20.4452138 44.6910160, 20.4495428 44.6880459, 20.4539572 44.6888231, 20.4529809 44.6911331, 20.4550753 44.6919188, 20.4534174 44.6929137, 20.4571253 44.6957696, 20.4570013 44.7008391, 20.4614601 44.7027894, 20.4646634 44.7018970, 20.4674388 44.7050131, 20.4753542 44.7039532, 20.4760757 44.7050260, 20.4802055 44.7033479, 20.4867635 44.7061539, 20.4983359 44.7022445, 20.5049892 44.7021663, 20.5071809 44.7071295, 20.5027682 44.7154832, 20.5028502 44.7217294, 20.5001912 44.7225288, 20.5007294 44.7251513, 20.5093727 44.7271542, 20.5316662 44.7248060, 20.5385861 44.7270519, 20.5390058 44.7329843, 20.5483761 44.7280993, 20.5513810 44.7308508, 20.5510751 44.7340860, 20.5483958 44.7345580, 20.5503614 44.7352316, 20.5509440 44.7434333, 20.5416617 44.7521169, 20.5358563 44.7553171, 20.5348919 44.7609694, 20.5393015 44.7624855, 20.5449353 44.7698750, 20.5490005 44.7708792, 20.5488362 44.7733456, 20.5647717 44.7649237, 20.5711431 44.7707818, 20.5772388 44.7711074, 20.5798915 44.7727751, 20.5852472 44.7808647, 20.5817268 44.7826053, 20.5823183 44.7845765, 20.5792147 44.7843299, 20.5777701 44.7872565, 20.5744279 44.7854098, 20.5740215 44.7886805, 20.5693220 44.7911579, 20.5655386 44.7906451, 20.5635444 44.7921747, 20.5598333 44.7901679, 20.5536143 44.7898282, 20.5502434 44.7909478, 20.5435002 44.8022967, 20.5424780 44.8073064, 20.5474459 44.8103678, 20.5530335 44.8102412, 20.5652728 44.8188428, 20.5738545 44.8279189, 20.5724006 44.8315147, 20.5776931 44.8371416, 20.5765153 44.8378971, 20.5863097 44.8427122, 20.5826128 44.8462544, 20.5762290 44.8486489, 20.5825139 44.8520894, 20.5953933 44.8552493, 20.6206689 44.8543410, 20.6212821 44.8560293, 20.6173687 44.8574761, 20.5961883 44.8615803, 20.5928447 44.8609861, 20.5911876 44.8626994, 20.6019440 44.8670619, 20.6196285 44.8673213, 20.6232109 44.8693710, 20.6164092 44.8815202, 20.6152606 44.8895682, 20.5777643 44.8860527, 20.5311826 44.8712209, 20.5230234 44.8646244, 20.5226088 44.8685278, 20.5187616 44.8654899, 20.5197414 44.8694015, 20.5132944 44.8687179, 20.5076686 44.8735038, 20.5065584 44.8670548, 20.4991594 44.8719635, 20.4938631 44.8734651, 20.4821047 44.8723679, 20.4737899 44.8677144, 20.4661802 44.8592493, 20.4594505 44.8560945, 20.4600397 44.8546034, 20.4650988 44.8535738, 20.4600110 44.8491680, 20.4623204 44.8477906, 20.4603705 44.8445375, 20.4711373 44.8342913, 20.4706338 44.8317839, 20.4498025 44.8343946, 20.4244846 44.8431449, 20.4138827 44.8526577, 20.3912248 44.8598333, 20.3749815 44.8683583, 20.3617778 44.8791076, 20.3436922 44.9103973, 20.3390650 44.9117584, 20.3011288 44.9426876, 20.2946156 44.9402419, 20.2960052 44.9381397, 20.2746476 44.9304194, 20.2703905 44.9345682, 20.2213764 44.9154621))"
)
# plot belgrade city limits
m = geopandas.GeoDataFrame(geometry=[toronto_poly], crs="epsg:4326").explore(name="Toronto", height=300, width=500)
# plot the points, just for demo purposes plot outcomes as different colors
#m = gdf.explore(m=m, column="riskLevel", cmap='black', name="points")


# add layer control so layers can be switched on / off
folium.LayerControl().add_to(m)

m

In [ ]:
coordinateDF = pd.DataFrame(DF, columns =["TCS #","Latitude", "Longitude"])
coordinateDF.head(5)
DF= coordinateDF.loc[:,["Latitude","Longitude"]]

In [ ]:
lon = [x for x in coordinateDF.Latitude]
lat = [x for x in coordinateDF.Longitude]
newMappedPoints = []
for idx, (lon,lat) in  enumerate(zip(lon,lat)):
  singlePoint = [idx,float(lon),float(lat)]
  newMappedPoints.append(singlePoint)


In [ ]:


d = {} 
for p,x,y in newMappedPoints:
    xs, ys = d.setdefault(p, [[],[]])
    xs.append(x), ys.append(y)

In [ ]:
# plot the points

f = plt.figure()
f.set_figwidth(10)
f.set_figheight(10)

for p, (xs, ys) in d.items():
    plt.scatter(xs, ys, label=p)

In [ ]:
labels= ['index','x','y']
df = pd.DataFrame.from_records(newMappedPoints,columns=labels)
df.head()